# Getting up to speed with Dask

## Part 2: Dask to the rescue!

We will do the same analysis as Part 1 with the same laptop-sized machine, except we will be using a rougly 10x larger dataset

AWS EC2 instance type: r5.xlarge (2 CPU, 16GB RAM)

Dask will work without intializing a distributed backend, but it's [recommended to do so](https://docs.dask.org/en/latest/setup/single-distributed.html) even on a laptop. To initialize, start a `Client` like so:

In [1]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:43779 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 16.25 GB


In [2]:
import pandas as pd
import numpy as np
import datetime
import s3fs
from pathlib import Path

# change this path if you changed in Part 0
data_path = Path('data')
seed = 42

# Load and explore data

In [4]:
taxi_dtypes = {
    'store_and_fwd_flag': str,
    'RatecodeID': 'float64',
    'VendorID': 'float64',
    'passenger_count': 'float64',
    'payment_type': 'float64',
}

def load_csv(f):
    """ Same as Part 1"""
    return pd.read_csv(
        f,
        dtype=taxi_dtypes, 
        parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [3]:
# new dask imports
from dask import delayed
import dask.dataframe as dd

## The hard way

We could parallelize our for-loop that blew up in Part 1 using [dask.delayed](https://docs.dask.org/en/latest/delayed.html).

NOTE: There is a better way below! This is just for educational purposes.

In [5]:
%%time

dfs = []
for f in data_path.iterdir():
    df = delayed(load_csv)(f)  # see the delayed() call
    dfs.append(df)
taxi = dd.from_delayed(dfs)  # from_delayed rather than pd.concat

CPU times: user 760 ms, sys: 66.2 ms, total: 826 ms
Wall time: 15.6 s


In [6]:
dfs

[Delayed('load_csv-e1644b79-c60d-49d2-89ba-77fd8dad7c98'),
 Delayed('load_csv-166a1a38-457b-4efd-9149-b157a401a7c9'),
 Delayed('load_csv-9f14bfe8-3bc6-47b5-9fe0-aa4535bfd561'),
 Delayed('load_csv-0420bf8e-cf27-4c62-a4e9-4528d27969e2'),
 Delayed('load_csv-dd5b9425-8ac9-4074-bd02-e6da01186940'),
 Delayed('load_csv-1680f615-aa4c-4228-be1b-4affb221147d'),
 Delayed('load_csv-8400332d-6d8f-4e9d-a79b-9575d4850cf8'),
 Delayed('load_csv-da8879cd-46e7-4cda-8f01-f85fad7be281'),
 Delayed('load_csv-19a6b346-53f7-419e-b2f2-8d74487fd11d'),
 Delayed('load_csv-dd9fc973-7409-45e8-8d4b-dcc711b80160'),
 Delayed('load_csv-f2f6c11d-1922-4412-94c9-1006ebde4fdb'),
 Delayed('load_csv-50b86535-d3ff-42b6-a462-c5d03d465c14')]

In [7]:
%%time
df0 = dfs[0].compute()

CPU times: user 2.21 s, sys: 2.43 s, total: 4.64 s
Wall time: 18.2 s


In [8]:
df0.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-12-01 00:26:58,2019-12-01 00:41:45,1.0,4.2,1.0,N,142,116,2.0,14.5,3.0,0.5,0.0,0.0,0.3,18.3,2.5
1,1.0,2019-12-01 00:12:08,2019-12-01 00:12:14,1.0,0.0,1.0,N,145,145,2.0,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
2,1.0,2019-12-01 00:25:53,2019-12-01 00:26:04,1.0,0.0,1.0,N,145,145,2.0,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
3,1.0,2019-12-01 00:12:03,2019-12-01 00:33:19,2.0,9.4,1.0,N,138,25,1.0,28.5,0.5,0.5,10.0,0.0,0.3,39.8,0.0
4,1.0,2019-12-01 00:05:27,2019-12-01 00:16:32,2.0,1.6,1.0,N,161,237,2.0,9.0,3.0,0.5,0.0,0.0,0.3,12.8,2.5


In [7]:
taxi

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=12,,,,,,,,,,,,,,,,,,
,float64,datetime64[ns],datetime64[ns],float64,float64,float64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
%%time
len(taxi)

CPU times: user 4.01 s, sys: 5.65 ms, total: 4.02 s
Wall time: 2min 20s


84399019

## Okay, let's do it the better way

`dask.dataframe.read_csv` has the same API as `pandas.dataframe.read_csv`

In [5]:
%%time

taxi = dd.read_csv(
    'data/yellow_tripdata_2019-*.csv',
    dtype=taxi_dtypes, 
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

CPU times: user 86.9 ms, sys: 15.8 ms, total: 103 ms
Wall time: 107 ms


In [12]:
%%time
len(taxi)

CPU times: user 4.16 s, sys: 223 ms, total: 4.38 s
Wall time: 2min 13s


84399019

<br>

Note the calls to `.compute()`. Dask executes _lazily_ meaning it won't do something unless you explicitly tell it to. Peek at the dashboard while these are executing to see the progress!

In [13]:
%%time
taxi.memory_usage(deep=True).sum().compute() / 1e9

CPU times: user 5.05 s, sys: 563 ms, total: 5.61 s
Wall time: 2min 56s


17.04023366

<br>
This takes a while

In [14]:
%%time
np.round(taxi.describe().compute(), 3).T

CPU times: user 2min 28s, sys: 8.04 s, total: 2min 36s
Wall time: 17min 19s


,count,mean,std,min,25%,50%,75%,max
VendorID,84152418.0,1.645,0.498,1.00,1.00,2.00,2.00,4.00
passenger_count,84152418.0,1.563,1.208,0.00,1.00,1.00,2.00,9.00
trip_distance,84399019.0,3.001,8.091,-37264.53,1.07,1.93,8.82,45977.22
RatecodeID,84152418.0,1.061,0.760,1.00,1.00,1.00,1.00,99.00
PULocationID,84399019.0,163.158,66.016,1.00,132.00,162.00,234.00,265.00
DOLocationID,84399019.0,161.353,70.251,1.00,116.00,163.00,236.00,265.00
payment_type,84152418.0,1.289,0.479,1.00,1.00,1.00,2.00,5.00
fare_amount,84399019.0,13.344,174.375,-1856.00,7.00,11.00,32.04,943274.80
extra,84399019.0,1.087,1.249,-60.00,0.00,1.00,3.00,535.38
mta_tax,84399019.0,0.495,0.067,-0.50,0.50,0.50,0.50,212.42


# Feature engineering

In [6]:
def make_features(df):
    """ Same code from Part 1 """
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df['pickup_year_seconds'] = (df.tpep_pickup_datetime - datetime.datetime(2019, 1, 1, 0, 0, 0)).dt.seconds
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['store_and_fwd_flag'] = (df.store_and_fwd_flag == 'Y').astype(int)
    df['VendorID'] = df.VendorID.fillna(-1)
    df['RatecodeID'] = df.RatecodeID.fillna(-1)

In [7]:
%%time

make_features(taxi)

CPU times: user 56.3 ms, sys: 151 µs, total: 56.5 ms
Wall time: 57.1 ms


In [17]:
%%time

taxi.head()

CPU times: user 102 ms, sys: 9.08 ms, total: 111 ms
Wall time: 2 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_year_seconds,pickup_week_hour
0,1.0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,0,151,239,1.0,...,0.0,0.3,9.95,NaN,1,1,0,46,2800,24
1,1.0,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,0,239,246,1.0,...,0.0,0.3,16.30,NaN,1,1,0,59,3587,24
2,2.0,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,0,236,236,1.0,...,0.0,0.3,5.80,NaN,4,51,13,48,49710,109
3,2.0,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,0,193,193,2.0,...,0.0,0.3,7.55,NaN,2,48,15,52,57145,63
4,2.0,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,0,193,193,2.0,...,0.0,0.3,55.55,NaN,2,48,15,56,57417,63


# Machine learning

In [8]:
# same as Part 1
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_minute', 
    'pickup_year_seconds',
    'pickup_week_hour', 
    'passenger_count',
]
categorical_feat = [
    'VendorID', 
    'RatecodeID', 
    'store_and_fwd_flag',
    'PULocationID',
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'total_amount'

In [9]:
# note the dask_ml imports rather than sklearn
from dask_ml.model_selection import train_test_split
from dask_ml.metrics import mean_squared_error
from dask_ml.xgboost import XGBRegressor

In [10]:
%%time

X_train, X_test, y_train, y_test = train_test_split(
    taxi[features], taxi[y_col], test_size=0.33, random_state=seed, shuffle=True)

CPU times: user 7.32 ms, sys: 0 ns, total: 7.32 ms
Wall time: 9.12 ms


<br>

Remember, nothing happens yet because of lazy evaluation. If you have the RAM, you can call `df.persist()` to avoid repeated CSV loading

(but if you had the RAM, you could do without Dask)

In [21]:
# X_train.persist()
# y_train.persist()

In [11]:
xgb = XGBRegressor(
    n_estimators=10, 
    max_depth=3, 
    learning_rate=0.1, 
    random_state=seed, 
    n_jobs=-1,
)

In [ ]:
%%time

_ = xgb.fit(X_train, y_train)

In [ ]:
%%time

# get test RMSE
preds = xgb.predict(X_test)
np.sqrt(mean_squared_error(preds, y_test.to_dask_array()))